**Load các file corpus và tải thư viện cần thiết**

In [ ]:
# Kết nối Google drive
from google.colab import drive
drive.mount('/content/drive')

import os
os.chdir("drive/My Drive/Colab Notebooks/Source_code/Corpus")

Mounted at /content/drive


In [ ]:
# Tải contractions và emoji: 
!pip install contractions
!pip install emoji

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 287 kB 4.2 MB/s 
     |████████████████████████████████| 106 kB 85.8 MB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 197 kB 4.2 MB/s 
  Created wheel for emoji: filename=emoji-2.0.0-py3-none-any.whl size=193022 sha256=ea8c106d7306f1fe34078a5d6a3beba6d3d3cec5f25ef7d261b59789b61e49a0
  Stored in directory: /root/.cache/pip/wheels/ec/29/4d/3cfe7452ac7d8d83b1930f8a6205c3c9649b24e80f9029fc38
Successfully built emoji


In [ ]:
# Import thư viện
import os
import re
import pandas as pd
import csv
import contractions
import nltk
import numpy as np
import emoji
from collections import Counter,defaultdict
from tqdm.notebook import tqdm
tqdm.pandas()

nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [ ]:
#Load các file corpus
df1 = pd.read_csv('eng-uk_web-public_2018_1M-sentences.txt',delimiter="\t",quoting=csv.QUOTE_NONE,names=['ids','sentences'])
df2 = pd.read_csv('eng-simple_wikipedia_2021_300K-sentences.txt',delimiter="\t",quoting=csv.QUOTE_NONE,names=['ids','sentences'])
df3 = pd.read_csv('eng-com_web-public_2018_1M-sentences.txt',delimiter="\t",quoting=csv.QUOTE_NONE,names=['ids','sentences'])
df4 = pd.read_csv('eng_wikipedia_2016_1M-sentences.txt',delimiter="\t",quoting=csv.QUOTE_NONE,names=['ids','sentences'])
df5 = pd.read_csv('eng_wikipedia_2012_1M-sentences.txt',delimiter="\t",quoting=csv.QUOTE_NONE,names=['ids','sentences'])
df6 = pd.read_csv('eng_wikipedia_2010_1M-sentences.txt',delimiter="\t",quoting=csv.QUOTE_NONE,names=['ids','sentences'])
df7 = pd.read_csv('eng_wikipedia_2007_1M-sentences.txt',delimiter="\t",quoting=csv.QUOTE_NONE,names=['ids','sentences'])
df8 = pd.read_csv('eng_news-typical_2016_1M-sentences.txt',delimiter="\t",quoting=csv.QUOTE_NONE,names=['ids','sentences'])
df9 = pd.read_csv('eng_newscrawl-public_2018_1M-sentences.txt',delimiter="\t",quoting=csv.QUOTE_NONE,names=['ids','sentences'])
df10 = pd.read_csv('eng_newscrawl_2018_1M-sentences.txt',delimiter="\t",quoting=csv.QUOTE_NONE,names=['ids','sentences'])
df11= pd.read_csv('eng_news_2020_1M-sentences.txt',delimiter="\t",quoting=csv.QUOTE_NONE,names=['ids','sentences'])

In [ ]:
# Gộp các file corpus thành một dataframe
df = pd.concat([df1,df2,df3,df4,df5,df6,df7,df8,df9,df10,df11])

**Tiền xử lý dữ liệu trước khi tokenize text**

In [ ]:
# Hàm xóa url trong text
def url_remover(data):
    return re.sub(r'(www\S+ ?)|(\w+\.com)|((http\S+))', '', data)

In [ ]:
# Hàm chuyển từ viết tắt sang từ tiêu chuẩn
# Ví dụ: you're -> you are
def contraction_replace(data):
  return contractions.fix(data)

In [ ]:
# Hàm loại bỏ nhiễu
# Xóa thể sở hữu cách trong từ tiếng anh là thể 's
# Xóa các ký tự đặt biết còn nằm lẫn trong từ như dấu chấm câu
# Trong hàm contraction.fix hiểu từ u. là viết tắt của you. Nên tất cả các từ
#   u.s đều bị hàm contraction.fix chuyển thành you.s, nên cần chuyển đổi lại
#   từ you.s -> u.s
def remove_noise(doc):
  doc = re.sub(r'(\'s|s\'|’s|s’|\(|\)|\[|\]|\{|\"|\'|\“|\’|\”|\‘|(\'.*))|\`|\—', '', doc)
  doc = re.sub(r'you.s',"u.s",doc,flags=re.IGNORECASE)
  return doc


In [ ]:
# Hàm tokenize text bằng thư viện nltk
def tokenize_doc(data):
  return nltk.word_tokenize(data)

In [ ]:
# Hàm xử lý dữ liệu trước khi tokenize và tokenize text
def preprocessing(doc):
  doc = str(doc)
  doc = doc.lower()
  doc = url_remover(doc)
  doc = emoji.demojize(doc).replace(':',' ')
  doc = contraction_replace(doc)
  doc = remove_noise(doc)
  tokens = tokenize_doc(doc)
  return tokens


In [ ]:
# Chạy hàm xử lý dữ liệu trước khi tokenize và toknize text
df['tokens'] = df.sentences.progress_apply(lambda x: preprocessing(x))

  0%|          | 0/1000000 [00:00<?, ?it/s]

**Xử lý dữ liệu sau khi Tokenize**

In [ ]:
# Hàm lọc ra những từ có tần suất xuất hiện ít hơn 5 lần trong corpus
#  và những ký tự đặc biệt
def get_rare_words_and_punctuation():
  n_tokens = 0
  word_freq = defaultdict(int)
  for sentence in df.tokens:
      for i in sentence:
          word_freq[i] += 1
          n_tokens +=1

  rare_words = []
  for key, value in word_freq.items():
    if value<5:
      rare_words.append(key) 

  rare_words.extend([token for token in '?!.,"#$%\'()*+-/:;<=>@[\\]^_`{|}~“”’‘&'])

  rare_words.extend(['--','—','——','•','«','……','––','—-','°','¬','·','▪','–-','—','−',
                     '―','›','-','..','»','+','*','‹','♦','₪','%N%','%$%','%^%','「','」'
                     ,'『','』','〈','〉','》','《','¿'])
  return Counter(rare_words)

rare_words_dict = get_rare_words_and_punctuation()

In [ ]:
#Hàm loại bỏ những từ chứa ký tự không nằm trong bảng mã ascii và không nằm trong
# tập rare word dict
def remove_rare_words_and_punctuation(sentence):
  return [word for word in sentence if word not in rare_words_dict and word.isascii()]

In [ ]:
# Chạy hàm xử lý dữ liệu sau khi tokenize
# Lưu vào Google drive
df['clean_tokens'] = df.tokens.progress_apply(lambda x: remove_rare_words_and_punctuation(x))
np.savetxt('preprocessed_corpus.txt',df.clean_tokens,fmt="%s")

  0%|          | 0/1000000 [00:00<?, ?it/s]

**In kết quả**

In [ ]:
word_freq = defaultdict(int)
n_tokens =0
for sentence in df.clean_tokens:
    for i in sentence:
        word_freq[i] += 1
        n_tokens +=1

In [ ]:
print(len(word_freq))
print(n_tokens)

296772
164834340
